In [1]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 992.7/992.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00


Question 1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:

What's the version of dlt that you installed?

In [2]:
!pip show dlt

Name: dlt
Version: 1.14.1
Summary: dlt is an open-source python-first scalable data loading library that does not require any backend to run.
Home-page: https://github.com/dlt-hub
Author: 
Author-email: "dltHub Inc." <services@dlthub.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: click, fsspec, gitpython, giturlparse, hexbytes, humanize, jsonpath-ng, orjson, packaging, pathvalidate, pendulum, pluggy, pytz, pyyaml, requests, requirements-parser, rich-argparse, semver, setuptools, simplejson, sqlglot, tenacity, tomlkit, typing-extensions, tzdata
Required-by: 


# dlt Resourse
For reading the FAQ data, we have this helper function:

Annotate it with @dlt.resource. We will use it when creating a dlt pipeline.

In [8]:
import dlt
import os
import requests


In [5]:
@dlt.resource(table_name="api_source_docs", max_table_nesting=0)
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

# Question 2. dlt pipeline
Now let's create a pipeline.

We need to define a destination for that. Let's use the qdrant one:

In [6]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant",
)

In this case, we tell dlt (and Qdrant) to create a folder with our data, and the name for it will be db.qdrant

Let's run it:

In [9]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-08-05 09:36:14.811297+00:00 and COMPLETED in 11.86 seconds with 4 steps.
Step extract COMPLETED in 0.36 seconds.

Load package 1754386575.3422406 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.13 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- api_source_docs: 948 row(s)

Load package 1754386575.3422406 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 10.86 seconds.
Pipeline zoomcamp_pipeline load step completed in 10.83 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /content/db.qdrant location to store data
Load package 1754386575.3422406 is LOADED and contains no failed jobs

Step run COMPLETED in 11.86 seconds.
Pipeline zoomcamp_pipeline load step completed in 10.83 seconds
1 load package(s) were loaded to destination qdrant and into

How many rows were inserted into the zoomcamp_data collection?

Look for "Normalized data for the following tables:" in the trace output.

In [10]:
import re

trace_output = """
Run started at 2025-08-05 09:36:14.811297+00:00 and COMPLETED in 11.86 seconds with 4 steps.
Step extract COMPLETED in 0.36 seconds.

Load package 1754386575.3422406 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.13 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- api_source_docs: 948 row(s)

Load package 1754386575.3422406 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 10.86 seconds.
Pipeline zoomcamp_pipeline load step completed in 10.83 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /content/db.qdrant location to store data
Load package 1754386575.3422406 is LOADED and contains no failed jobs

Step run COMPLETED in 11.86 seconds.
Pipeline zoomcamp_pipeline load step completed in 10.83 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /content/db.qdrant location to store data
Load package 1754386575.3422406 is LOADED and contains no failed jobs
"""

match = re.search(r"- api_source_docs: (\d+) row\(s\)", trace_output)
if match:
    rows_inserted = match.group(1)
    print(f"Number of rows inserted into api_source_docs: {rows_inserted}")
else:
    print("Could not find the number of rows inserted.")

Number of rows inserted into api_source_docs: 948


# Question 3. Embeddings

When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the meta.json file created in the target folder. During the data insertion process, a folder named db.qdrant will be created, and the meta.json file will be located inside this folder.

In [24]:
qdrant_destination

<dlt.destinations.qdrant(destination_type='qdrant', staging_dataset_name_layout='%s_staging', enable_dataset_name_normalization=True, info_tables_query_threshold=1000, local_dir='/content', pipeline_name='zoomcamp_pipeline', pipeline_working_dir='/var/dlt/pipelines/zoomcamp_pipeline', qd_path='db.qdrant', dataset_separator='_', embedding_batch_size=32, embedding_parallelism=0, upload_batch_size=64, upload_parallelism=1, upload_max_retries=3, model='BAAI/bge-small-en')>

In [31]:
import re

metadata_string = "<dlt.destinations.qdrant(destination_type='qdrant', staging_dataset_name_layout='%s_staging', enable_dataset_name_normalization=True, info_tables_query_threshold=1000, local_dir='/content', pipeline_name='zoomcamp_pipeline', pipeline_working_dir='/var/dlt/pipelines/zoomcamp_pipeline', qd_path='db.qdrant', dataset_separator='_', embedding_batch_size=32, embedding_parallelism=0, upload_batch_size=64, upload_parallelism=1, upload_max_retries=3, model='BAAI/bge-small-en')>"

match = re.search(r"model='(.*?)'", metadata_string)
if match:
    model_name = match.group(1)
    print(f"The embedding model used is: {model_name}")
else:
    print("Could not find the embedding model name in the metadata string.")

The embedding model used is: BAAI/bge-small-en
